In [0]:
from pyspark.sql import SparkSession, functions as F, types as T, Window
from delta.tables import DeltaTable
import os
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window

In [0]:
from pyspark.sql import types as T

# Nome da tabela de destino no Catálogo Unity
catalog_table = "production.refined.d_hoteis"

# Verifica se a tabela já existe antes de tentar criá-la
if not spark.catalog.tableExists(catalog_table):
    print(f"A tabela {catalog_table} não existe. Criando a estrutura...")
    
    # Define o schema para a dimensão de hotéis, seguindo o padrão pk/sk
    schema = T.StructType([
        # Chave primária (hash da chave de negócio, ex: hotel_id)
        T.StructField("pk_hotel", T.StringType(), False), 
        # Chave substituta (surrogate key) sequencial e única
        T.StructField("sk_hotel", T.LongType(), False),
        # Chave de negócio original para referência
        T.StructField("hotel_id", T.IntegerType(), True),
        
        # Atributos descritivos do hotel
        T.StructField("nome_hotel", T.StringType(), True),
        T.StructField("estrelas", T.IntegerType(), True),
        T.StructField("comodidades", T.StringType(), True),

        # Metadados de controle (SCD - Slowly Changing Dimensions)
        T.StructField("start_date", T.DateType(), True),
        T.StructField("update_date", T.DateType(), True)
    ])
    
    # Cria um DataFrame vazio com o schema definido
    df_empty = spark.createDataFrame([], schema)

    # Cria a tabela Delta gerenciada no catálogo
    (
        df_empty.write
        .format("delta")
        .saveAsTable(catalog_table)
    )

    print(f"Tabela Delta '{catalog_table}' criada com sucesso.")
else:
    print(f"A tabela '{catalog_table}' já existe.")

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from pyspark.sql.window import Window
from delta.tables import DeltaTable

# --- Parâmetros ---
catalog_table = "production.refined.d_hoteis"
source_table = "production.trusted.tb_hoteis" 

print("Iniciando processo de merge para a tabela:", catalog_table)

# --- 1. Carregar Tabela de Destino ---
try:
    delta_table = DeltaTable.forName(spark, catalog_table)
    df_dim_existente = delta_table.toDF()
    is_initial_load = df_dim_existente.count() == 0
except Exception as e:
    # Captura exceção específica para tabela não encontrada
    if "TABLE_OR_VIEW_NOT_FOUND" in str(e):
        is_initial_load = True
        print(f"Tabela {catalog_table} não encontrada. Assumindo carga inicial.")
    else:
        raise e

# --- 2. Carregar e Preparar Dados de Origem ---
# Seleciona as colunas da origem que correspondem à dimensão de destino.
df_source = (
    spark.read.table(source_table)
    .select(
        "hotel_id", 
        "nome_hotel", 
        "estrelas", 
        "comodidades"
    )
    .dropDuplicates(["hotel_id"])
    .withColumn(
        "pk_hotel",
        F.sha2(F.col("hotel_id").cast(T.StringType()), 256)
    )
)

# --- 3. Identificar e Preparar Apenas os Novos Registros ---
if is_initial_load:
    df_novos_hoteis = df_source
else:
    # Isola apenas os hotéis que são realmente novos (não existem no destino)
    df_novos_hoteis = df_source.join(
        df_dim_existente.select("pk_hotel"),
        on="pk_hotel",
        how="left_anti"
    )

# Se não houver novos hotéis, encerra a execução para economizar recursos.
if not is_initial_load and df_novos_hoteis.count() == 0:
    print("Nenhum novo hotel para adicionar. Processo concluído.")
    # dbutils.notebook.exit("Nenhum novo hotel para adicionar.") # Descomente para parar o notebook

# --- 4. Gerar Chaves Surrogadas (SKs) Apenas para os Novos Registros ---
if is_initial_load:
    last_id = 0
else:
    last_id_row = df_dim_existente.agg(F.max("sk_hotel")).collect()
    last_id = last_id_row[0][0] if last_id_row and last_id_row[0][0] is not None else 0

window = Window.orderBy("pk_hotel")
df_com_novas_sks = (
    df_novos_hoteis
    .withColumn("sk_hotel", (F.row_number().over(window) + last_id).cast(T.LongType()))
    .withColumn("start_date", F.current_date())
    .withColumn("update_date", F.lit(None).cast(T.DateType()))
)

# --- 5. Criar o DataFrame Final para o MERGE ---
# Une a fonte completa com as novas SKs (que só existem para os novos registros)
df_final_source = df_source.join(
    df_com_novas_sks.select("pk_hotel", "sk_hotel", "start_date", "update_date"),
    "pk_hotel",
    "left"
)

# --- 6. Executar a Operação de MERGE ---
# Condição para atualizar: apenas se dados descritivos do hotel mudarem.
update_condition = """
    target.nome_hotel <> source.nome_hotel OR
    target.estrelas <> source.estrelas OR
    target.comodidades <> source.comodidades
"""

(
    delta_table.alias("target")
    .merge(
        df_final_source.alias("source"),
        "target.pk_hotel = source.pk_hotel"
    )
    .whenMatchedUpdate(
        condition=update_condition,
        set={
            "nome_hotel": F.col("source.nome_hotel"),
            "estrelas": F.col("source.estrelas"),
            "comodidades": F.col("source.comodidades"),
            "update_date": F.current_date()
        }
    )
    .whenNotMatchedInsert(
        values={
            "pk_hotel": F.col("source.pk_hotel"),
            "sk_hotel": F.col("source.sk_hotel"),
            "hotel_id": F.col("source.hotel_id"),
            "nome_hotel": F.col("source.nome_hotel"),
            "estrelas": F.col("source.estrelas"),
            "comodidades": F.col("source.comodidades"),
            "start_date": F.col("source.start_date"),
            "update_date": F.col("source.update_date")
        }
    )
).execute()

print(f"Merge/upsert concluído com sucesso na tabela: {catalog_table}")

In [0]:
%sql
select * from production.refined.d_hoteis